In [483]:
import os 
import time 
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import io
from io import BytesIO
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import pathlib
import cv2

In [568]:
DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
wd.get(search_url.format(q='dog'))
body = wd.find_element_by_tag_name("body")
    



In [570]:
len(new)

884

In [635]:
def scrape(search_name,web_driver=wd):
    
    DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
    wd.get(search_url.format(q=search_name))
    body = wd.find_element_by_tag_name("body")
    
    for i in range(100):
        time.sleep(.5)
        try:    
            body.send_keys(Keys.PAGE_DOWN)
        except Exception:
            pass
    
    click = wd.find_element_by_class_name('mye4qd')
    click.click()

    for i in range(100):
        time.sleep(.5)
        try:
            body.send_keys(Keys.PAGE_DOWN)
        except Exception:
            pass 
        
    data = wd.find_elements_by_class_name("rg_i.Q4LuWd")
    links1 = [data[i].get_attribute('src') for i in range(len(data))]
    links2 = [data[i].get_attribute('data-src') for i in range(len(data))]
    test = links1 + links2

    new = []
    for i in test:
        try:
            if type(i) != None:
                if i[0:4] == 'data':
                    i = i.replace("data:image/jpeg;base64,","")
                    if i[-2:] != '==':
                        new_edit = new + '=='
                        i = (Image.open(BytesIO(base64.b64decode(i)))).resize((150,150))
                        new.append(i)
                    else:
                        i = (Image.open(BytesIO(base64.b64decode(i)))).resize((150,150))
                        new.append(i)
                if i[0:4] == 'http':
                    i = requests.get(i)
                    i = Image.open(BytesIO(i.content))
                    new.append(i)
            if type(i) == None:
                pass
        except Exception:
            pass
    return new
    wd.quit()

    
def save(list_of_images,class_of_image,valid_ratio):

    valid = float(valid_ratio * float(len(list_of_images)))
    valid_data = []
    training_data = [] 
    int_valid = int(valid)
    
    for i in range(len(list_of_images)):
        if i < int_valid:
            valid_data.append(list_of_images[i])
        else:
            training_data.append(list_of_images[i])
    
    class_type = str(class_of_image)
    training_path = 'Images/training/' + class_type
    valid_path = 'Images/validation/' + class_type

    try:
        os.mkdir('Images')
    except Exception:
        pass
    try:
        os.mkdir('Images/training')
    except Exception:
        pass
    try:
        os.mkdir('Images/validation')
    except Exception:
        pass
    try:
        os.mkdir(training_path)
    except Exception:
        pass
    try:
        os.mkdir(valid_path)
    except Exception:
        pass
   
    validation_index = 0 
    for i in valid_data:
        try:
            i.save(valid_path + '/' + class_type + str(validation_index) + '.jpeg')
            validation_index += 1 
        except Exception:
            pass
    training_index=0
    for i in training_data:
        try:
            i.save(training_path + '/' + class_type + str(training_index) + '.jpeg')
            training_index += 1 
        except Exception:
            pass
    return 
   




def augment(data,class_type):    
    
    training_path = 'Images/training/' + class_type
    for i in range(len(data)):
        index = 0 
        image = np.expand_dims(data[i],axis=0)

        aug_data_generator = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.35,
        height_shift_range=0.175,
        horizontal_flip=True,
        vertical_flip = True)
    
        if len(image.shape) == 4:

            for batch in aug_data_generator.flow(image, 
                           batch_size=5,
                          save_to_dir= training_path,
                           save_prefix= class_type, 
                           save_format='jpeg'):
                    index += 1 
                    if index == 5:
                        break
        else: 
            pass
    


In [649]:
len(dog)

1479

In [636]:
dog = scrape('dog')

In [637]:
cat = scrape('cat')

In [648]:
len(cat)

1192

In [638]:
save_dog = save(dog,'dog',.2)

In [639]:
save_cat = save(cat,'cat',.2)

In [640]:
aug_dog = augment(dog,'dog')

In [641]:
aug_cat = augment(cat,'cat')

In [647]:
training = 'Images/training'
validation = 'Images/validation'

train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
        training,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=len(dog),
        class_mode='categorical') 

valid_datagen = ImageDataGenerator()
validation_generator = valid_datagen.flow_from_directory(
        validation,
        target_size=(150, 150),
        batch_size=len(cat),
       class_mode='categorical')

Found 11794 images belonging to 2 classes.
Found 532 images belonging to 2 classes.


In [645]:
len(train_generator)

369

In [548]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3),padding='same', input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Conv2D(32, (3, 3),padding='same',))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same',))

model.add(Conv2D(64, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#model.save_weights('first_try.h5')

In [650]:

model.fit_generator(
        train_generator,
        steps_per_epoch=1,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50


InvalidArgumentError:  input depth must be evenly divisible by filter depth: 3 vs 150
	 [[node sequential_1/activation_1/Relu (defined at <ipython-input-650-403ed6f955fe>:7) ]] [Op:__inference_train_function_1166]

Function call stack:
train_function
